In [165]:
import textworld
import glob
import jericho
from jericho import FrotzEnv
from collections import defaultdict
import pandas as pd
import nltk

# GAME_FOLDER = "/Users/SXH1M01/Data/the-large-game-collection"
GAME_FOLDER = "/Users/SXH1M01/GitHub/z-machine-games/jericho-game-suite"
files = glob.glob(f"{GAME_FOLDER}/*")
len(files)

57

In [191]:
import nltk
from nltk.corpus import wordnet
syns = wordnet.synsets("table")
for syn in syns:
    print(syn.lemmas()[0].name())

table
table
table
mesa
table
board
postpone
table


In [207]:
wordnet.synsets("kitchen")[0].hypernyms()

[Synset('room.n.01')]

In [229]:
def get_hypernyms(word):
    word = word.lower().strip()
    syns = wordnet.synsets(word)
    hypernyms = set()
    for syn in syns:
        for h in syn.hypernyms():
            for l in h.lemmas():
                hypernyms.add(l.name().replace("_"," "))
    return hypernyms

def get_hyponyms(word):
    word = word.lower().strip()
    syns = wordnet.synsets(word)
    hyponyms = set()
    for syn in syns:
        for h in syn.hyponyms():
            for l in h.lemmas():
                hyponyms.add(l.name().replace("_"," "))
    return hyponyms

In [230]:
get_hypernyms("table")

{'arrange',
 'array',
 'article of furniture',
 'assemblage',
 'delay',
 'fare',
 'furniture',
 'gathering',
 'piece of furniture',
 'plateau',
 'set',
 'tableland'}

In [231]:
get_hyponyms("table")

{"Lord's table",
 'Parsons table',
 'actuarial table',
 'altar',
 'billiard table',
 'board',
 'booth',
 'breakfast table',
 'calendar',
 'call',
 'card table',
 'cocktail table',
 'coffee table',
 'communion table',
 'conference table',
 'console',
 'console table',
 'contents',
 'correlation table',
 'council board',
 'council table',
 'counter',
 'desk',
 'dining table',
 'dresser',
 'dressing table',
 'drop-leaf table',
 'file allocation table',
 'gaming table',
 'gueridon',
 'hold',
 'kitchen table',
 'operating table',
 'pedestal table',
 'periodic table',
 'pier table',
 'ping-pong table',
 'pingpong table',
 'platen',
 'pool table',
 'probate',
 'reprieve',
 'respite',
 'snooker table',
 'stand',
 'statistical table',
 'suspend',
 'table of contents',
 'table-tennis table',
 'tea table',
 'toilet table',
 'training table',
 'trestle table',
 'vanity',
 'work table',
 'worktable'}

In [234]:
list(get_hyponyms("room"))[:5]

['court', 'sea room', 'bathroom', 'standing room', 'ballroom']

In [ ]:
k = wordnet.synset('kitchen.n.01')

In [203]:
k = wordnet.synset('kitchen.n.01')
k.hypernyms(),k.hyponyms()

([Synset('room.n.01')],
 [Synset('galley.n.03'), Synset('galley.n.04'), Synset('kitchenette.n.01')])

In [4]:
# !python -m spacy download en_core_web_sm

In [5]:
game2pairs = dict()
for fname in files:
    jenv = FrotzEnv(fname)
    
    walkthrough = None
    initial_observation, info = jenv.reset()
    try:
        walkthrough = jenv.get_walkthrough()
    except:
        continue
    if not walkthrough:
        continue
        
    pairs = [("start",initial_observation.strip())]
    for act in walkthrough:    
        observation, reward, done, info = jenv.step(act)
        pairs.append((act.strip().lower(),observation.strip()))
    game2pairs[fname.split("/")[-1]] = pairs
len(game2pairs)

56

In [6]:
len(game2pairs), len(files)

(56, 57)

In [12]:
action_tally = defaultdict(int)
action_obs = defaultdict(list)
for game, pairs in game2pairs.items():
    for act, obs in pairs:
        action_tally[act] += 1
        action_obs[act].append(obs)

In [56]:
stop_words = set("a,an,the,you,your,have".split(","))

def remove_stopwords(tokens):
    return [tok for tok in tokens if tok not in stop_words and not "(" in tok and not ")" in tok]
    
inventory_items = defaultdict(int)
for obs in action_obs["i"] + action_obs["inventory"]:
    obs = obs.lower().strip()
    if "not " in obs or "nothing" in obs or "$" in obs:
        continue
    lines = obs.split(":")[-1].split("\n")
    for line in lines:
        line = line.strip()
        if not line:
            continue
        tokens = remove_stopwords(line.split(" "))
        if len(tokens) >= 10:
            continue
        inventory_items[" ".join(tokens)] += 1
inventory_items

defaultdict(int,
            {'sheet of red paper': 2,
             "three comb's teeth": 2,
             "lady's tortoise-shell comb": 3,
             'bronze-hilted dagger': 3,
             'half porcelain mask': 3,
             'armor and silks': 3,
             'kitestring': 2,
             'tasseled fan': 2,
             'red paper kite with no string': 1,
             'tweed jacket': 1,
             'floppy felt hat': 1,
             'incredibly long scarf': 1,
             'yo-yo': 1,
             'tardis key': 1,
             'id card': 1,
             'towel': 3,
             "hitchhiker's guide": 4,
             'flowerpot': 3,
             'number twelve asteroid paint chipper': 3,
             'circuit board': 3,
             'pair of tweezers': 3,
             'jacket fluff': 3,
             'pair of hypersonic pliers': 3,
             'ionic diffusion rasp': 3,
             'flathead screwdriver': 4,
             'toothbrush': 4,
             'pocket fluff': 4,
          

In [77]:
# location2obs["breakwater"]

["Breakwater\nYou are picking your way across the breakwater's bare, rocky spine, where the road has diminished to little more than a pair of dusty ruts. A hundred yards to the northeast, at the breakwater's far end, the lonely stone tower of Anchorhead's lighthouse stands vigil against the ocean.\n\nThe sea thrashes against the shore with storm-driven fury."]

In [78]:
locations = defaultdict(int)
location2obs = defaultdict(list)
loc_actions = "n,e,s,w,ne,nw,se,sw,north,south,east,west,northeast,northwest,southeast,southwest,u,up,d,down".split(",")
for act in loc_actions:
    for obs in action_obs[act]:
        locn = obs.lower().strip().split("\n")[0]
        if not locn or '"' in locn or "?" in locn or "!" in locn or not locn.isalpha():
            continue
        tokens = remove_stopwords(locn.split(" "))
        if len(tokens) >= 10:
            continue
        location = " ".join(tokens)
        locations[location] += 1
        location2obs[location].append(obs)
# sorted(locations.items(), key = lambda tpl: -tpl[1])

In [80]:
len([(loc,cnt) for loc,cnt in locations.items() if cnt > 1])

134

In [129]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging

ie_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz")
def info_extract(sent):
    return ie_predictor.predict(
        sentence=sent
    )

In [169]:
text = """
A bottle is sitting on the table.
The glass bottle contains:
  A quantity of water
"""

In [170]:
ie = info_extract(text)
words = ie["words"]

In [171]:
relations = []
for v in ie["verbs"]:    
    tags = v["tags"]
    relations.append([(wd,tag) for wd,tag in zip(words,tags) if tag != "O"])
relations

[[('is', 'B-V')],
 [('A', 'B-ARG1'),
  ('bottle', 'I-ARG1'),
  ('sitting', 'B-V'),
  ('on', 'B-ARG2'),
  ('the', 'I-ARG2'),
  ('table', 'I-ARG2')],
 [('The', 'B-ARG2'),
  ('glass', 'I-ARG2'),
  ('bottle', 'I-ARG2'),
  ('contains', 'B-V'),
  ('A', 'B-ARG1'),
  ('quantity', 'I-ARG1'),
  ('of', 'I-ARG1'),
  ('water', 'I-ARG1')]]

In [173]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

Noun phrases: ['A bottle', 'the table', 'The glass bottle', 'A quantity', 'water']
Verbs: ['sit', 'contain']


In [174]:
rows = []
for token in doc:
#     print(token.text.ljust(30), token.pos_, token.tag_, token.dep_,
#             token.shape_, token.is_alpha, token.is_stop)
    if not token.text.strip():
        continue
    rows.append({
        'text': token.text,
        'pos': token.pos_,
        'tag': token.tag_,
        'dep': token.dep_
    })
pd.DataFrame(rows)

,text,pos,tag,dep
0,A,DET,DT,det
1,bottle,NOUN,NN,nsubj
2,is,AUX,VBZ,aux
3,sitting,VERB,VBG,ROOT
4,on,ADP,IN,prep
5,the,DET,DT,det
6,table,NOUN,NN,pobj
7,.,PUNCT,.,punct
8,The,DET,DT,det
9,glass,NOUN,NN,compound


In [124]:
# qa_predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/commonsenseqa.2020-07-08.tar.gz")

In [125]:
# qa_predictor.predict(prefix="Which of these are found in a kitchen?", 
#     alternatives = [
#     "a table",
#     "a lamp",
#     "a car"
# ])

In [235]:
for obs in location2obs['kitchen'][0:5]:
    print(obs)
    print("*" * 80)

Kitchen
The kitchen a spacious and hi-tech. Reminds you of Dotcom-era extravegance. There are alot of cupboards, topped by a marble counter. Doors lead east and south.
********************************************************************************
Kitchen
You're in the kitchen, by the looks of things. Some broken cabinets are on the wall. Light from partially boarded-up windows lets you see the rotting wood floors. Rooms lie to the north and west.

Strangely untouched and undamaged, a GUE-Automatic oven is sitting closed in a corner of the room.
********************************************************************************
Kitchen
A bottle is sitting on the table.
The glass bottle contains:
  A quantity of water
There is a brown sack here.
The brown sack contains:
  A lunch
********************************************************************************
Kitchen
A bottle is sitting on the table.
The glass bottle contains:
  A quantity of water
There is a brown sack here.
The brown sa

In [236]:
list(locations)[0:5]

['bedroom', 'hall', 'antechamber', 'connection', 'corridor']

In [70]:
action_tally["down"]

58

In [237]:
len(action_tally)

4137

In [239]:
sorted(action_tally.items(), key = lambda tpl : -tpl[1])[0:10]

[('n', 1015),
 ('e', 983),
 ('s', 974),
 ('w', 971),
 ('z', 493),
 ('wait', 406),
 ('d', 383),
 ('u', 331),
 ('ne', 201),
 ('sw', 185)]

In [245]:
def print_solution(game):
    for state, obs in game2pairs[game]:
        print(state)
        print(obs.strip())
        print("*" * 80)

print_solution("zork1.z5")

start
Copyright (c) 1981, 1982, 1983 Infocom, Inc. All rights reserved.
ZORK is a registered trademark of Infocom, Inc.
Revision 88 / Serial number 840726

West of House
You are standing in an open field west of a white house, with a boarded front door.
There is a small mailbox here.
********************************************************************************
n
North of House
You are facing the north side of a white house. There is no door here, and all the windows are boarded up. To the north a narrow path winds through the trees.
********************************************************************************
n
Forest Path
This is a path winding through a dimly lit forest. The path heads north-south here. One particularly large tree with some low branches stands at the edge of the path.
********************************************************************************
u
Up a Tree
You are about 10 feet above the ground nestled among some large branches. The nearest branch above you 

In [246]:
print_solution("advent.z5")

start
Welcome to Adventure!
(Please type HELP for instructions and information.)


ADVENTURE
The Interactive Original
By Will Crowther (1976) and Don Woods (1977)
Reconstructed in three steps by:
Donald Ekman, David M. Baggett (1993) and Graham Nelson (1994)
[In memoriam Stephen Bishop (1820?-1857): GN]

Release 9 / Serial number 060321 / Inform v6.31 Library 6/11 S

At End Of Road
You are standing at the end of a road before a small brick building. Around you is a forest. A small stream flows out of the building and down a gully.
********************************************************************************
e
Inside Building
You are inside a building, a well house for a large spring.

There are some keys on the ground here.

There is tasty food here.

There is a shiny brass lamp nearby.

There is an empty bottle here.
********************************************************************************
get all
set of keys: Taken.
tasty food: Taken.
brass lantern: Taken.
small bottle: Tak

In [8]:
sorted(game2pairs.keys())

['905.z5',
 'acorncourt.z5',
 'advent.z5',
 'adventureland.z5',
 'afflicted.z8',
 'anchor.z8',
 'awaken.z5',
 'balances.z5',
 'ballyhoo.z3',
 'curses.z5',
 'cutthroat.z3',
 'deephome.z5',
 'detective.z5',
 'dragon.z5',
 'enchanter.z3',
 'enter.z5',
 'gold.z5',
 'hhgg.z3',
 'hollywood.z3',
 'huntdark.z5',
 'infidel.z3',
 'inhumane.z5',
 'jewel.z5',
 'karn.z5',
 'library.z5',
 'loose.z5',
 'lostpig.z8',
 'ludicorp.z5',
 'lurking.z3',
 'moonlit.z5',
 'murdac.z5',
 'night.z5',
 'omniquest.z5',
 'partyfoul.z8',
 'pentari.z5',
 'planetfall.z3',
 'plundered.z3',
 'reverb.z5',
 'seastalker.z3',
 'sherlock.z5',
 'snacktime.z8',
 'sorcerer.z3',
 'spellbrkr.z3',
 'spirit.z5',
 'temple.z5',
 'theatre.z5',
 'trinity.z4',
 'tryst205.z5',
 'weapon.z5',
 'wishbringer.z3',
 'yomomma.z8',
 'zenon.z5',
 'zork1.z5',
 'zork2.z5',
 'zork3.z5',
 'ztuu.z5']

In [49]:
# for act, obs in log:
#     print("Act:", act)
#     print(obs.strip())
#     print()

In [145]:
FOLDER = "/Users/SXH1M01/Data/club-floyd-gameplay-transcripts/*.html"
files = glob.glob(FOLDER)

In [193]:
all_pairs = defaultdict(list)
cnt = -1
for fname in files:
    with open(fname) as f:
        lines = f.readlines()
    if not lines:
        continue
    unique = set()
    cnt +=1
    for l in lines:
        new_pairs = [p.strip().split("[ACTION]") for p in l.split("[STATE]") if p]
        new_pairs = [(state.strip(),act.strip().lower()) for state,act in new_pairs]
        for pair in new_pairs:
            state, act = pair
            if act == "begin journey":
                cnt += 1
                # point pairs to a new list
                unique = set()
                all_pairs[cnt].append(pair)
                unique.add(pair)
            elif pair not in unique:
                all_pairs[cnt].append(pair)
                unique.add(pair)

pairs = []
for pp in all_pairs.values():
    if not pp:
        continue
    new_pairs = []
    pairs.append(new_pairs)
    for i in range(len(pp)-1):
        state, act = pp[i]
        next_state, next_act = pp[i+1]
        new_pairs.append((act, next_state))
len(pairs)

420

In [198]:
pairs[0]

[('begin journey', 'Would you like to restore a saved game?'),
 ('north',
  'Mother\'s voice weighs you down like a bag of sand. You grasp Teddy harder as you stare hypnotized at the huge finger stabbing the air not more than half an inch from your nose. You shrink back. "Behave now you damned little devil. You little beast! If I hear a single word from out here when I talk to your dear aunt, a single word... There is no need for her to finish as the tightness of her puffy lips and the steel in her eyes tells you all you need to know. Your enormous and absolutely terrifying mother stares intently at you for an excruciatingly long moment before she turns around. She stomps away towards the next room, and then she stops. Just when you think that she is going to move on, she turns and comes back. Your heart nearly escapes your body when she smiles. convinced now that your mother is an alien, or something unimaginably worse, you try to back away from her. There is no way out though. In one